In [ ]:
# ============================================
# 📊 MODEL 2 - POLYNOMIAL REGRESSION (QUADRATIC)
# ============================================

print("\n" + "=" * 70)
print("📊 MODEL 2: POLYNOMIAL REGRESSION (QUADRATIC)")
print("=" * 70)
print("\nFormula: temperature_change = β₀ + β₁ × year + β₂ × year² + ε")
print("Purpose: Capture acceleration/deceleration of warming trend")

# Entrenar modelo polinomial
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

# Predicciones
y_train_pred_poly = model_poly.predict(X_train_poly)
y_test_pred_poly = model_poly.predict(X_test_poly)

# Métricas
train_r2_poly = r2_score(y_train, y_train_pred_poly)
test_r2_poly = r2_score(y_test, y_test_pred_poly)
train_rmse_poly = np.sqrt(mean_squared_error(y_train, y_train_pred_poly))
test_rmse_poly = np.sqrt(mean_squared_error(y_test, y_test_pred_poly))
train_mae_poly = mean_absolute_error(y_train, y_train_pred_poly)
test_mae_poly = mean_absolute_error(y_test, y_test_pred_poly)

print("\n📈 Model Coefficients:")
print("-" * 70)
print(f"Intercept (β₀):     {model_poly.intercept_:.2f} °C")
print(f"Linear term (β₁):   {model_poly.coef_[0]:.6f} °C/year")
print(f"Quadratic term (β₂): {model_poly.coef_[1]:.9f} °C/year²")

# Interpretación del término cuadrático
if model_poly.coef_[1] > 0:
    print(f"\n✅ ACCELERATION detected: β₂ > 0")
    print(f"   → Warming rate is INCREASING over time")
    
    # Calcular la aceleración en décadas
    # Derivada: d(temp)/d(year) = β₁ + 2*β₂*year
    accel_1961 = model_poly.coef_[0] + 2*model_poly.coef_[1]*1961
    accel_2022 = model_poly.coef_[0] + 2*model_poly.coef_[1]*2022
    
    print(f"   → Rate in 1961: {accel_1961:.6f} °C/year ({accel_1961*10:.4f} °C/decade)")
    print(f"   → Rate in 2022: {accel_2022:.6f} °C/year ({accel_2022*10:.4f} °C/decade)")
    print(f"   → Acceleration: {(accel_2022-accel_1961)*10:.4f} °C/decade increase")
else:
    print(f"\n❌ DECELERATION: β₂ < 0")
    print(f"   → Warming rate is DECREASING over time (unlikely)")

print("\n📊 Model Performance:")
print("-" * 70)
print(f"Training Set:")
print(f"  R² Score:  {train_r2_poly:.4f}")
print(f"  RMSE:      {train_rmse_poly:.4f} °C")
print(f"  MAE:       {train_mae_poly:.4f} °C")
print(f"\nTest Set:")
print(f"  R² Score:  {test_r2_poly:.4f}")
print(f"  RMSE:      {test_rmse_poly:.4f} °C")
print(f"  MAE:       {test_mae_poly:.4f} °C")

# Comparación con modelo simple
print("\n🔍 Comparison with Simple Linear Model:")
print("-" * 70)
print(f"R² improvement (test): {(test_r2_poly - test_r2_simple):.4f}")
print(f"RMSE improvement (test): {(test_rmse_simple - test_rmse_poly):.4f} °C")

if test_r2_poly > test_r2_simple:
    improvement = ((test_r2_poly - test_r2_simple) / test_r2_simple) * 100
    print(f"✅ Polynomial model is {improvement:.2f}% better (R²)")
else:
    print(f"⚠️  Simple model performs better (avoid overfitting)")

# Visualización comparativa
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Comparación de ajustes
all_years = modeling_data['year'].values
X_all_simple = all_years.reshape(-1, 1)
X_all_poly = np.column_stack([all_years, all_years**2])
y_pred_all_simple = model_simple.predict(X_all_simple)
y_pred_all_poly = model_poly.predict(X_all_poly)

axes[0, 0].scatter(modeling_data['year'], modeling_data['temp_mean'], 
                   alpha=0.6, s=50, color='black', label='Actual Data', zorder=3)
axes[0, 0].plot(all_years, y_pred_all_simple, 
                'b--', linewidth=2, label='Linear Model', alpha=0.7)
axes[0, 0].plot(all_years, y_pred_all_poly, 
                'r-', linewidth=2, label='Quadratic Model')
axes[0, 0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 0].axvline(x=train_cutoff, color='purple', linestyle=':', alpha=0.5)
axes[0, 0].set_title('Model Comparison: Linear vs Quadratic', fontweight='bold', fontsize=12)
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Temperature Change (°C)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Residuales del modelo polinomial
residuals_train_poly = y_train - y_train_pred_poly
residuals_test_poly = y_test - y_test_pred_poly

axes[0, 1].scatter(y_train_pred_poly, residuals_train_poly, 
                   alpha=0.6, s=50, color='blue', label='Train')
axes[0, 1].scatter(y_test_pred_poly, residuals_test_poly, 
                   alpha=0.6, s=50, color='green', label='Test')
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_title('Polynomial Model: Residuals vs Fitted', fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('Fitted Values (°C)')
axes[0, 1].set_ylabel('Residuals (°C)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Comparación de residuales (Simple vs Poly)
residuals_simple_all = modeling_data['temp_mean'].values - model_simple.predict(X_all_simple)
residuals_poly_all = modeling_data['temp_mean'].values - model_poly.predict(X_all_poly)

axes[1, 0].scatter(modeling_data['year'], residuals_simple_all, 
                   alpha=0.5, s=30, color='blue', label='Linear Model')
axes[1, 0].scatter(modeling_data['year'], residuals_poly_all, 
                   alpha=0.5, s=30, color='red', marker='x', label='Quadratic Model')
axes[1, 0].axhline(y=0, color='gray', linestyle='--', linewidth=2)
axes[1, 0].set_title('Residuals Over Time: Model Comparison', fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Residuals (°C)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Histograma comparativo de residuales
axes[1, 1].hist(residuals_simple_all, bins=20, alpha=0.5, color='blue', 
                edgecolor='black', label=f'Linear (std={np.std(residuals_simple_all):.3f})')
axes[1, 1].hist(residuals_poly_all, bins=20, alpha=0.5, color='red', 
                edgecolor='black', label=f'Quadratic (std={np.std(residuals_poly_all):.3f})')
axes[1, 1].axvline(x=0, color='gray', linestyle='--', linewidth=2)
axes[1, 1].set_title('Distribution of Residuals', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Residual (°C)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/home/jovyan/reports/figures/regression_model2_polynomial.png', 
            dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Key Insight:")
if model_poly.coef_[1] > 0 and test_r2_poly > test_r2_simple:
    print("   The quadratic model confirms warming ACCELERATION and provides better fit.")
    print("   This has critical implications for 2030+ projections.")
else:
    print("   Linear model may be sufficient. Acceleration is not strongly supported.")